In [1]:
import numpy as np
import pandas as pd
import glob
import ipyparallel as p
import os
import ntpath
import ProgressBar as pb
from pymail import alert
import time

In [2]:
exname = 'smex8_sen'
method = '8'
site = 'Nr1'

In [3]:
files = glob.glob('/Volumes/data/RHESSys_out/Nwt/%s/*_basin.daily'%(exname))

In [4]:
files[-10:]

['/Volumes/data/RHESSys_out/Nwt/smex8_sen/smex8_sen_99994_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex8_sen/smex8_sen_99995_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex8_sen/smex8_sen_99996_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex8_sen/smex8_sen_99997_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex8_sen/smex8_sen_99998_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex8_sen/smex8_sen_99999_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex8_sen/smex8_sen_9999_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex8_sen/smex8_sen_999_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex8_sen/smex8_sen_99_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex8_sen/smex8_sen_9_basin.daily']

In [5]:
params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

In [6]:
params.head()

,MAP,PeakSWE,SMR,TpeakSWE,combo,idx,rd,ref,scenidx,sd
0,6.165935,0.41366,0.007447,180.0,1,0,0.003764,2010,0,4.003764
1,6.165935,0.41366,0.007447,180.0,2,0,0.037645,2010,1,4.037645
2,6.165935,0.41366,0.007447,180.0,3,0,0.075289,2010,2,4.075289
3,6.165935,0.41366,0.007447,180.0,4,0,0.150578,2010,3,4.150578
4,6.165935,0.41366,0.007447,180.0,5,0,0.301157,2010,4,4.301157


In [7]:
files = pd.DataFrame({'basin_daily':files})

In [8]:
def getIDX(fl):
    return int(fl.split('_')[-2])

In [9]:
files['scenidx'] = files.basin_daily.map(getIDX)

In [10]:
params = pd.merge(params,files, on= 'scenidx')

In [11]:
from rhessys import snowmelt_experiment as smex

In [12]:
params['Tnosnow'] = params.TpeakSWE+np.ceil(params.PeakSWE/params.SMR)

In [13]:
params['date_peakSWE'] = params.apply(smex.DOPEAKSWE2cal,axis=1)
params['date_ONS'] =params.apply(smex.DONS2cal,axis=1)

/Users/barnhatb/python/bin/rhessys/snowmelt_experiment.py:12: FutureWarning: pandas.core.datetools.timedelta is deprecated. Please use datetime.timedelta instead.
  td = pd.datetools.timedelta(x.TpeakSWE-1)
/Users/barnhatb/python/bin/rhessys/snowmelt_experiment.py:20: FutureWarning: pandas.core.datetools.timedelta is deprecated. Please use datetime.timedelta instead.
  td = pd.datetools.timedelta(x.Tnosnow-1)


In [16]:
c = p.Client()
view = c.load_balanced_view()

In [17]:
c.ids

[0, 1, 2, 3, 4, 5, 6, 7]

In [18]:
len(params)

150000

In [19]:
dr = pd.date_range('2009-10-1','2010-9-30',freq='D')

In [20]:
%px dr = pd.date_range('2009-10-1','2010-9-30',freq='D')

In [21]:
def processSM(bd,idx,strt,nd):

    # read in the basin daily data
    data = pd.read_table(bd,delimiter=' ')

    data.index = pd.DatetimeIndex(dr)

    data['ET'] = data.evap + data.trans
    trans = data.trans.sum()
    evap = data.evap.sum()
    ET = data.ET.sum() # compute total ET
    Q = data.streamflow.sum() # compute total streamflow 
    Qbf = data.baseflow.sum() # compute total baseflow
    Qrf = data['return'].sum() # compute total returnflow,p
    sm = data.snowmelt.sum()
    rech = data.recharge.sum() # compute total recharge
    sat = data['%sat_area'].mean() # compute the mean saturated area
    p = data.precip.sum() # compute water year precip
    rz_storage = data.rz_storage.mean() # root zone storage
    rz_drainage = data.rz_drainage.mean() # root zone drainage 
    unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
    unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
    sat_def = data.sat_def.mean() # saturation deficite (mm)
    sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?
    gwStore = data['gw.storage'].sum() # gw store
    gwFlux = data['gw.Qout'].mean()
    ETRate = data.ET.mean()
    
    # create full output object
    full = (idx,trans,evap,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,
            unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate)
    
    data = data.loc[strt:nd] # crop the data to the melt season
    
    T = data.trans.sum() # compute totoal transpiration
    E = data.evap.sum() # compute total evaporation
    ET = data.ET.sum() # compute total ET
    Q = data.streamflow.sum() # compute total streamflow 
    Qbf = data.baseflow.sum() # compute total baseflow
    Qrf = data['return'].sum() # compute total returnflow,p
    sm = data.snowmelt.sum()
    rech = data.recharge.sum() # compute total recharge
    sat = data['%sat_area'].mean() # compute the mean saturated area
    p = data.precip.sum() # compute water year precip
    rz_storage = data.rz_storage.mean() # root zone storage
    rz_drainage = data.rz_drainage.mean() # root zone drainage 
    unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
    unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
    sat_def = data.sat_def.mean() # saturation deficite (mm)
    sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?
    gwStore = data['gw.storage'].sum() # gw store
    gwFlux = data['gw.Qout'].mean()
    ETRate = data.ET.mean()
    
    melt = (idx,T,E,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,
            unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate)
    
    return full,melt

In [22]:
idx = 3
print params.basin_daily[idx]
processSM(params.basin_daily[idx],params.scenidx[idx],params.date_peakSWE[idx],params.date_ONS[idx])

/Volumes/data/RHESSys_out/Nwt/smex8_sen/smex8_sen_3_basin.daily


((3,
  73.546043999999995,
  249.83572900000001,
  323.38177300000001,
  264.61180699999994,
  249.65519899999998,
  14.956607999999999,
  261.86143900000002,
  0.0,
  546.92161399999998,
  69.83511210137019,
  0.06792593150684931,
  1813.6498518602655,
  20.148218726027398,
  1935.5508087561616,
  4057.7585088520555,
  155.50990100000001,
  2083.7049819999997,
  0.18750979726027409,
  0.8859774602739737),
 (3,
  5.0148570000000001,
  23.176295,
  28.191151999999999,
  0.17590299999999998,
  0.17590299999999998,
  0.0,
  14.103344,
  0.0,
  1.179243,
  69.23514405263164,
  0.01195259649122807,
  1857.2379975087706,
  0.0,
  1979.8182803859665,
  4150.562432280701,
  17.742263000000001,
  8.5664680000000004,
  0.002917614035087719,
  0.4945816140350877))

In [23]:
#res = view.map(processSM,params.basin_daily[0:1000],params.scenidx[0:1000],params.date_peakSWE[0:1000],params.date_ONS[0:1000])
res = view.map(processSM,params.basin_daily,params.scenidx,params.date_peakSWE,params.date_ONS)

In [ ]:
prog = pb.ProgressBar(len(params))

In [ ]:
while res.ready() == False:
    prog.animate_ipython(res.progress)
    time.sleep(2)

[********         21%                  ]  31188 of 150000 complete

In [ ]:
res.done()

In [ ]:
full,melt = zip(*res.result())

## Merge processing results with parameters dataset

In [ ]:
idx,trans,evap,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate = zip(*full)

params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,trans, evap, ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,
        sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate]
keys = ['scenidx','trans','evap','ET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage',
        'unsat_storage','unsat_drainage','sat_def','sat_def_z','sm','gwStore','gwFlux','ETRate']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

params = pd.merge(params,proc,on='scenidx') # join on file index number
params.dropna(inplace=True) # drop missing values

params.to_hdf('./data/%s_%s_full.hdf'%(site,exname),'df',format='fixed',complevel=5,complib='bzip2',fletcher32=True)

In [ ]:
idx,T,E,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate = zip(*melt)

params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,T,E,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,
        sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate]
keys = ['scenidx','T','E','ET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage',
        'unsat_storage','unsat_drainage','sat_def','sat_def_z','sm','gwStore','gwFlux','ETRate']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

params = pd.merge(params,proc,on='scenidx') # join on file index number
params.dropna(inplace=True) # drop missing values

params.to_hdf('./data/%s_%s_melt.hdf'%(site,exname),'df',format='fixed',complevel=5,complib='bzip2',fletcher32=True) # 